In [9]:
from tqdm import tqdm_notebook as tqdm
import preprocessor as p
import numpy as np

In [10]:
%run ../twitter15/twitter15_text_processing.ipynb

In [11]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim
import torch.nn.functional as F

In [12]:
import pickle as pkl
from collections import defaultdict
import pandas as pd
import os
import numpy as np
import json
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [13]:
from collections import Counter
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [14]:
import matplotlib.pyplot as plt
import random

In [15]:
class Node:
    def __init__(self,uid,tid,time_stamp,label):
        self.children = {}
        self.childrenList = []
        self.num_children = 0
        self.tid = tid
        self.uid = uid
        self.label = label
        self.time_stamp = time_stamp
    
    def add_child(self,node):
        if node.uid not in self.children:
            self.children[node.uid] = node
            self.num_children += 1
        else:
            self.children[node.uid] = node
        self.childrenList = list(self.children.values())

In [16]:
class Tree:
    def __init__(self,root):
        self.root = root
        self.tweet_id = root.tid
        self.uid = root.uid
        self.height = 0
        self.nodes = 0
    
    def show(self):
        queue = [self.root,0]
        
        while len(queue) != 0:
            toprint = queue.pop(0)
            if toprint == 0:
                print('\n')
            else:
                print(toprint.uid,end=' ')
                queue += toprint.children.values()
                queue.append(0)
                
    def insertnode(self,curnode,parent,child):
        if curnode.uid == parent.uid:
            curnode.add_child(child)
            return 1

        elif parent.uid in curnode.children:
            s = self.insertnode(curnode.children[parent.uid],parent,child)
            return 2
        else:
            for node in curnode.children:
                s = self.insertnode(curnode.children[node],parent,child)
                if s == 2:
                    break

In [9]:
def loadPklFileNum(datapath,incSize,fileNum):
    
    with open(datapath+str(incSize)+'inc_'+str(fileNum)+'.pickle', 'rb') as handle:
        twitTrees = pkl.load(handle)
    return twitTrees

In [10]:
def loadTreeFilesOfIncrement(datapath,incSize):
    twittertrees = {}
    
    files = [x for x in os.listdir(t15Datapath) if str(incSize)+'inc' in x]
    
    for file in tqdm(files):
        with open(datapath+file,'rb') as handle:
            partialTrees = pkl.load(handle)
        twittertrees.update(partialTrees)
        
    return twittertrees

In [11]:
t15Datapath = '../twitter15/pickledTrees/'
# twitter15_trees = loadPklFileNum(t15Datapath,20,1)

In [12]:
twitter15_trees = loadTreeFilesOfIncrement(t15Datapath,20)

100%|██████████| 9/9 [01:22<00:00,  9.19s/it]


In [17]:
%run ../twitter15/userdata_parser.ipynb


100%|██████████| 1000/1000 [00:00<00:00, 25946.98it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27


100%|██████████| 33/33 [02:06<00:00,  3.84s/it]

 40%|███▉      | 100571/253378 [00:24<00:36, 4151.43it/s]


 83%|████████▎ | 211203/253378 [00:51<00:10, 4125.36it/s]


100%|██████████| 253378/253378 [01:01<00:00, 4094.19it/s]


In [18]:
for key in tqdm(userVects):
    userVects[key] = userVects[key].float()

userVects = defaultdict(lambda:torch.tensor([1.1100e+02, 1.5000e+01, 0.0000e+00, 7.9700e+02, 4.7300e+02, 0.0000e+00,
        8.3326e+04, 1.0000e+00]),userVects)


100%|██████████| 253378/253378 [00:00<00:00, 325551.51it/s]


In [23]:
userVects[0].requires_grad

False

In [15]:
%run ./textEncoders.ipynb

I0107 10:05:24.364788 139830510675712 file_utils.py:39] PyTorch version 1.3.0 available.
I0107 10:05:24.399865 139830510675712 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [16]:
%run ./temporal_tree_model.ipynb

In [17]:
if torch.cuda.is_available():
    device = 'cuda:2'
    device = 'cpu'
else:
    device = 'cpu'

In [18]:
labelMap = {}
labelCount = 0
for label in list(twitter15_labels.values()):
    if label not in labelMap:
        labelMap[label] = labelCount
        labelCount += 1
labelMap

{'false': 0, 'true': 1, 'unverified': 2, 'non-rumor': 3}

In [26]:
epochs = 10
X = []
y = []
X_text = []

for tid in twitter15_trees:
        if tid in twitter15_trees and tid in twitter15_labels:
            X.append(tuple((twitter15_trees[tid],twitter15_text[tid])))
            y.append(labelMap[twitter15_labels[tid]])
            X_text.append(twitter15_text[tid])
            
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=2018)

NameError: name 'twitter15_trees' is not defined

In [27]:
X_text

[]

In [38]:
class jointModel(nn.Module):
    def __init__(self, treeEncoderType, textEncoderType, pretrain, textparams, treeparams, X, y, device):
        super(jointModel, self).__init__()
        if textEncoderType == 'rnn':
            self.textEncoderModel = TextEncoder(textEncoderType,textparams,X,y,device)
            textparams['hidden_dim'] = textparams['hidden_dim']*self.textEncoderModel.textEncoder.numDirs
            
        if textEncoderType == 'bert':
            self.textEncoderModel = BertTextEncoder(textEncoderType,{},X,y,device)
            textparams['hidden_dim'] = 768
        
        if textEncoderType == 'attn':
            self.textEncoderModel = AttentionTextEncoder(textEncoderType,textparams,X,y,device)
            textparams['hidden_dim'] = textparams['hidden_dim']*self.textEncoderModel.textEncoder.numDirs*self.textEncoderModel.seq_dim
        
        if pretrain:
            self.textEncoderModel.trainModel()
            self.textEncoderModel.textEncoder.load_state_dict(self.textEncoderModel.optimalParams)
            
        if treeEncoderType == 'standard':
            self.treeEncoderModel = treeEncoder(treeparams['cuda'],treeparams['in_dim'],treeparams['mem_dim'],treeparams['userVects'],treeparams['labels'],treeparams['labelMap'],treeparams['criterion'],device)
        if treeEncoderType == 'decay':
            self.treeEncoderModel = decayTreeEncoder(treeparams['cuda'],treeparams['in_dim'],treeparams['mem_dim'],treeparams['userVects'],treeparams['labels'],treeparams['labelMap'],treeparams['criterion'],device)
        
        mem_dim = treeparams['mem_dim'] + textparams['hidden_dim']
        
        self.fc = nn.Linear(mem_dim,4)    
            
    def forward(self,tree,text):
        treeVec = self.treeEncoderModel(tree)
        treeVec = treeVec[0][1].reshape(-1)
        
        self.textEncoderModel.textEncoder = self.textEncoderModel.textEncoder.to('cpu')
        textVec = self.textEncoderModel(text)
        textVec = textVec.reshape(-1)
#         print(treeVec.shape)
#         print(textVec.shape)
        combVec =  torch.cat((treeVec,textVec))
#         combVec = textVec
        out = self.fc(combVec)
        return out

In [41]:
def trainModel(model,modelname):
#     optimizer = torch.optim.Adagrad(model.treeEncoderModel.parameters(),0.01)
    
#     bertoptimizer = AdamW(model.textEncoderModel.parameters(),
#                   lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
#                   eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
#                 )

    optimizer = torch.optim.Adagrad(model.parameters(),0.01)
    
    criterion = nn.CrossEntropyLoss()

    maxAcc = 0
    count = 0
    
    for i in range(10):
        for treeSet, text in tqdm(x_train):
            tree = treeSet[-1]
            optimizer.zero_grad()
            
            pred = model(tree.root,text)
            
            label = Variable(torch.tensor(labelMap[treeSet[0].root.label]).reshape(-1).to(device))
            loss = criterion(pred.reshape(1,4),label)
#             print(loss)
    
            if count % 20 == 0:
#                 print('opt')
                loss.backward()
                optimizer.step()
        
        preds = []
        labels = []

        allLabels = []
        allPreds = []
        
        for valSet, text in tqdm(x_test):
            finalTree = valSet[-1]

            predicted = model(finalTree.root,text)
            preds.append(predicted)
    #         print(predicted)
            predicted =  torch.softmax(predicted,0)
            predicted = torch.max(predicted, 0)[1].cpu().numpy().tolist()

            labels.append(labelMap[finalTree.root.label])

            allLabels.append(labelMap[finalTree.root.label])
            allPreds.append(predicted)

        predTensor = torch.stack(preds)
        labelTensor = torch.tensor(labels).to(device)

        print(allLabels,allPreds)

        loss = criterion(predTensor.reshape(-1,4), labelTensor.reshape(-1))

        cr = classification_report(allLabels,allPreds,output_dict=True)
        cr['loss'] = loss.item()
        cr['Acc'] = accuracy_score(allLabels,allPreds,)

        if cr['Acc'] > maxAcc:
            maxAcc = cr['Acc']
#             torch.save({'state_dict': model.state_dict()}, './jointlyTrainedResults/'+modelname+'.pth')

        print('loss: ',cr['loss'])
        print(cr['Acc'])

        with open('./jointlyTrainedResults/'+modelname+'json', 'a') as fp:
            json.dump(cr, fp)
            fp.write('\n')

treeparams = {
    'cuda': torch.cuda.is_available(),
    'in_dim':8,
    'mem_dim':100,
    'userVects':userVects,
    'labels':twitter15_labels,
    'labelMap':labelMap,
    'criterion':nn.CrossEntropyLoss()
}

textparams = {
    'embedding_dim':256,
    'hidden_dim': 50,
    'output_dim':4,
    'num_layers':1,
    'bidir':True,
    'rnnType':'gru'
}

treeTypes = ['standard','decay']
textTypes = ['bert']
pretrainTypes = [True,False]

treeTypes = ['standard','decay']
textTypes = ['bert']
pretrainTypes = [True]

for textType in textTypes:
    for treeType in treeTypes:
        for pretrainType in pretrainTypes:
            model = jointModel(treeType,textType,pretrainType,textparams,treeparams,X_text,y,device)
            model = model.to(device)
            modelname = textType+'_'+treeType+'-tree_pretrain-'+str(pretrainType)
            print(modelname)
            trainModel(model,modelname)

In [ ]:
treeparams = {
    'cuda': torch.cuda.is_available(),
    'in_dim':8,
    'mem_dim':100,
    'userVects':userVects,
    'labels':twitter15_labels,
    'labelMap':labelMap,
    'criterion':nn.CrossEntropyLoss()
}

textparams = {
    'embedding_dim':256,
    'hidden_dim': 50,
    'output_dim':4,
    'num_layers':1,
    'bidir':True,
    'rnnType':'gru'
}

treeTypes = ['standard']
textTypes = ['rnn']
pretrainTypes = [False,]
bidirTypes = [False]
rnnTypes = ['gru']

for textType in textTypes:
    for treeType in treeTypes:
        for pretrainType in pretrainTypes:
            for rnnType in rnnTypes:
                for bidirType in bidirTypes:
                    textparams['rnnType'] = rnnType
                    textparams['bidirType'] = bidirType
                    
                    model = jointModel(treeType,textType,pretrainType,textparams,treeparams,X_text,y,device)
                    model = model.to(device)
                    modelname = textType+'_'+treeType+'-tree_pretrain-'+str(pretrainType)+'_'+rnnType+'bidir-'+str(bidirType)
                    print(modelname)
                    trainModel(model,modelname)